# Creación del dataset DIAxI en formato COCO

## 1. Solucionar problemas con el *encoding* de los archivos

Un problema con el que nos encontramos es que tanto el escaneo del archivo de Abuelas como el etiquetado colaborativo fue realizado en distintos sistemas operativos que utilizan diferentes tipos de *encoding* para el nombre de los archivos. Para poder construir el dataset tratamos de convertir todo a UTF-8 conservando la mayor cantidad de archivos posible.

> - Requiere instalar `convmv` en la distribución GNU/Linux.
> - Se asume que los archivos JSON se encuentran en la carpeta `jsons` y el archivo completo de imágenes en `dataset`.

In [ ]:
%%bash
# convmv -f UTF-8 -t ISO-8859-1 -r jsons --fixdouble --notest
# convmv -f ISO-8859-1 -t UTF-8 -r jsons --notest
# convmv -f UTF-8 -t ISO-8859-1 -r dataset --fixdouble --notest
# convmv -f ISO-8859-1 -t UTF-8 -r dataset --notest


Luego para cada archivo JSON se copia su correspondiente archivo TIFF a la carpeta `fixed`.

In [ ]:
import os
from shutil import copyfile

os.makedirs("fixed/images", exist_ok=True)
os.makedirs("fixed/jsons", exist_ok=True)

lost = []
for j in os.listdir("jsons"):
    img = j.replace(".json", ".tif")

    try:
        copyfile(f"dataset/{img}", f"fixed/images/{img}")
        copyfile(f"jsons/{j}", f"fixed/jsons/{j}")

    except FileNotFoundError:
        lost.append(j)

In [ ]:
len(lost)  # Perdimos 51 archivos en el camino :(

## 2. Enderezado de las imágenes

In [ ]:
import os
import math
from typing import Tuple, Union

import cv2
import numpy as np
from deskew import determine_skew

THRESHOLD = 10  # límite de 10 grados

def rotate(image: np.ndarray, angle: float, background: Union[int, Tuple[int, int, int]]) -> np.ndarray:
    old_width, old_height = image.shape[:2]
    angle_radian = math.radians(angle)
    width = abs(np.sin(angle_radian) * old_height) + abs(np.cos(angle_radian) * old_width)
    height = abs(np.sin(angle_radian) * old_width) + abs(np.cos(angle_radian) * old_height)

    image_center = tuple(np.array(image.shape[1::-1]) / 2)
    rot_mat = cv2.getRotationMatrix2D(image_center, angle, 1.0)
    rot_mat[1, 2] += (width - old_width) / 2
    rot_mat[0, 2] += (height - old_height) / 2
    
    return cv2.warpAffine(image, rot_mat, (int(round(height)), int(round(width))), borderValue=background)

for i in os.listdir("fixed/images/"):
    image = cv2.imread(f"fixed/images/{i}")
    grayscale = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    angle = determine_skew(grayscale)

    if abs(angle) < THRESHOLD:
        rotated = rotate(image, angle, (0, 0, 0))
        cv2.imwrite(f"fixed/images/{i}", rotated)

## 3. Creación del archivo JSON principal

El formato COCO (*Common Objects in Context*) consta de un archivo JSON y un directorio de imágenes. Vamos a construir el archivo JSON de acuerdo a las especificaciones del formato, que es bastante simple y sólo consta de 4 llaves: `images`, `categories`, `annotations` e `info`.

In [ ]:
import json
from PIL import Image

jsons = sorted(os.listdir("fixed/jsons"))
tiffs = sorted(os.listdir("fixed/images"))

Primero hacemos verificamos si todos los JSONs tienen una imagen que les corresponda.

In [ ]:
for i in range(len(tiffs)):
    assert jsons[i].replace(".json", ".tif") == tiffs[i]

Creamos la llave `info` con metadata.

In [ ]:
info = {"contributor": "Lionel Messi",
        "date_created": "2022-12-18 10:10:10.101010",
        "description": "",
        "url": "",
        "version": "1.0",
        "year": 2023
       }

Creamos la llave `annotations`.

Para ello leemos cada archivo JSON y buscamos recursivamente todas las llaves que tengan el nombre `bounding_box`. El nombre de la llave que esté por encima de cada *bounding box* será el nombre de la *clase* (`category_id`) a la que pertenece.

In [ ]:
def item_generator(json_input, lookup_key, parent_key=None):
    if isinstance(json_input, dict):
        for k, v in json_input.items():
            if k == lookup_key:
                yield parent_key, v

            else:
                yield from item_generator(v, lookup_key, k)

    elif isinstance(json_input, list):
        for i, item in enumerate(json_input):
            if isinstance(item, (dict, list)):
                for result in item_generator(item, lookup_key, parent_key=f"{parent_key}"):
                    yield result

            elif item == lookup_key:
                yield parent_key, item

In [ ]:
images = []
annotations = []

keys = ["Diario", "Fecha", "Notas", "Página", "Copete", "Cuerpo", "Destacado", "Epígrafe", "Firma", "Fotografía", "Título", "Volanta"]
class2id = dict(zip((keys), range(len(keys))))

for i,j in enumerate(jsons):
    # `images` key
    img = {}
    im = Image.open(f"fixed/images/{tiffs[i]}")
    w, h = im.size

    img["id"] = i
    img["file_name"] = f"images/{tiffs[i]}"
    img["width"] = w
    img["height"] = h
    images.append(img)

    # `annotations` key
    with open(f"fixed/jsons/{j}", "r") as f:
        data = json.load(f)

        gen = item_generator(data, "bounding_box")
        for g in gen:
            try:
                cid = class2id[g[0]]

            except KeyError:
                continue

            try:
                x, y, w, h = g[1].values()

            except ValueError:
                continue

            ann = { "area": w*h,
                    "bbox": [x, y, w, h],
                    "category_id": cid,
                    "id": 0,
                    "ignore": 0,
                    "image_id": i,
                    "iscrowd": 0,
                    "segmentation": [[x, y, x+w, y, x+w, y+h, x, y+h]],
                }

            annotations.append(ann)

Enumeramos las anotaciones entre `0..N`.

In [ ]:
n = 0
for ann in annotations:
    ann["id"] = n
    n += 1

Creamos la llave `categories`.

In [ ]:
categories = [{"id": v, "name": k} for (k,v) in class2id.items()]

Nosotros consideramos que el nombre `Imagen` es más adecuado que `Fotografía`.

In [ ]:
categories[9]["name"] = "Imagen"

Finalmente guardamos el resultado en un nuevo archivo JSON.

In [ ]:
result = {"images": images, 
          "categories": categories, 
          "annotations": annotations, 
          "info": info
          }

In [ ]:
with open("fixed/result.json", "w") as f:
    json.dump(result, f, indent=2)

## 4. Filtrar clases (opcional)

Utilizamos el script `catfilter.py` para conservar las anotaciones pertenecientes al subconjunto de clases que queremos usar en nuestro modelo.

In [ ]:
%%bash 
python scripts/catfilter.py -i fixed/result.json -o fixed/result_filtered.json -c 'Copete,Cuerpo,Destacado,Epígrafe,Imagen,Título,Volanta'

## 5. Descartar imágenes

Posteriormente se guardan las imágenes anotadas con `pyodi` y se procede a una inspección ocular de los resultados. Posteriormente se borran a mano las imágenes cuyas *bounding boxes* no coinciden con los elementos que pretenden localizar.

In [ ]:
%%bash
pyodi paint-annotations fixed/result_filtered.json fixed fixed/painted

Actualizamos el archivo JSON para descartar las imágenes que no pasaron el filtro "humano".

In [ ]:
img_ok = [f"images/{i}".replace("_result.tif", ".tif") for i in sorted(os.listdir("fixed/painted"))]

with open("fixed/result_filtered.json", "r") as f:
    filtered = json.load(f)

final_images = []
for img in filtered["images"]:
    if img["file_name"] in img_ok:
        final_images.append(img)

filtered["images"] = final_images

with open("fixed/result_final.json", "w") as f:
    json.dump(filtered, f, indent=2, sort_keys=True)

## 6. Fusionar datasets (opcional)

Con `pyodi` se pueden fusionar distintos sets de datos en formato COCO. Pero primero es necesario arreglar los JSON para mapear las IDs de las imágenes entre `0..N`.

In [ ]:
%%bash
# python scripts/imgidfix.py -i <archivo json> <archivo json de salida>

In [ ]:
%%bash
# pyodi coco merge <archivo json 1> <archivo json 2> <archivo json de salida>

## 7. Comprimir el set de datos

Finalmente comprimimos el set de datos para su distribución

In [ ]:
%%bash
mkdir -p dataset-diaxi-coco
cp -r fixed/images dataset-diaxi-coco
cp fixed/result_final.json dataset-diaxi-coco/result.json
cd dataset-diaxi-coco
zip -r ../dataset-diaxi-coco.zip images result.json